# Computer Assignemet 1 - Search Algorithms
## Nazanin Yousefian
### 810197610

## Abstract
### In this project, we want to solve a problem using different search strategies: Uninformed search and informed search. we implement BFS and IDS for uninformed search. A* and weighted A* are two informed search strategies we tend to implement. All these algorithms are optimal wich means they find the path with lowest cost but they differ in time complexity and mamory consumption. 

## Introduction
### We all know the popular *Snake* game. We have a map and a snake of length 1. We have to eat all the apples in the map. In this project we find to find the shortest path to reach goal (eating all apples) with different strategies. We should keep track of each state which is created after snake moves. In limited time for each test case and algorithm, the solution must be printed.

### project Description
we need to have the information of each state. Based on the map, the coordinates of remained apples and the length of our snake and coordinates of each point of its body we should distinguish states. we define a class named *Node* to achieve this goal. *snake* is a list which starts with head of snake and ends with its tail. *move* is a character showing what move of snake led it to this state. *par* is parent node of this node. *num* is the number of apples remained(apples with score of 2 are counted twice). *food1* is a list including coordinates of each apple with 1 score and it is sorted. *food2* is a list like *food1* for 2-score apples. *State* is a tuple and shows properties of our  map. It itsel has 3 tuples inside it: *snake*, *food1*, and *food2*. Knowing state of each node prevents us from checking duplicate states and results in faster solution. 
fields *cost* and *h* are g(n) and h(n) of each node wich are used in A* algorithm.
Our first state is the beginning of the game having a snake of length one and all apples. For recognizing it, a field *first* is added and its value for first state is *'first'*. For all other nodes its an empty string. The goal state is reaches when all apples are eaten and the game finishes. 


In [597]:
import queue
import heapq
import time
from collections import defaultdict
class Node:
    def __init__(self, snake, move, parent, num, food1, food2, first = ''):
        self.snake = snake
        self.move = move
        self.par = parent
        self.first = first
        self.num = num
        self.food1 = food1
        self.food2 = food2
        self.state = None
        self.cost = 0
        self.h = 0
    def __lt__(self, other):
        return self.cost < other.cost

    def set_cost(self, cost, h):
        self.cost = cost
        self.h = h


In [598]:
def test_goal(num):
    return num == 0

In [599]:
def get_input(filename, snake, food1, food2):
    file1 = open(filename, 'r')
    Lines = file1.readlines()
    [y, x] = [int(x) - 1 for x in Lines[0].split(',')]
    head = [int(x) for x in Lines[1].split(',')]
    snake.append([head[0], head[1]])
    num = int(Lines[2])
    size = 0
    for i in range(num):
        temp = [int(x) for x in Lines[i + 3].split(',')]
        if temp[2] == 1:
            food1.append((temp[0], temp[1]))
            size += 1
        else:
            food2.append((temp[0], temp[1]))
            size += 2
    return size, y, x

In [600]:
def initial_state(snake, size, food1, food2):
    node = Node(snake, '', [], size, food1, food2, 'first')
    node.state = (tuple(map(tuple, snake)), tuple(food1), tuple(food2))
    return node

The function below checks if there is an apple in coornidates of snake's head. It returns a tuple(a boolean and number of remained apples).

In [601]:
def eat_apple(head, food1,food2, num):
    eat = False
    if (head[0], head[1]) in food1:
        food1.remove((head[0], head[1]))
        num -= 1
        eat = True
    elif (head[0], head[1]) in food2:
        food2.remove((head[0], head[1]))
        food1.append((head[0], head[1]))
        food1.sort()
        eat = True
        num -= 1
    return eat, num

*find_move* indicates all possible moves for the snake from this state. if the length of snake is 1, it can go to any direction. Otherwise, it calls another function named *adjacent*. 
In *adjacent* function for every move(up, down, left, and right) we check two conditions:
First, the second pat of the snake's body should not be in the direction that snake wants to move. Second, The tial of the snake should not be in the direction that snake wants to move. 
All the possible moves of stored in a set. 

In [602]:
def find_move(snake, move, y, x):
    if len(snake) == 1:
        move.update('r', 'l', 'u', 'd')
    else:
        adjacent(snake, y, x, move)

In [603]:
def adjacent(snake, y, x, move):

    #up
    if snake[0][0] == 0:
        if not ((snake[1][0] == y) and (snake[0][1] == snake[1][1])):
            move.add('u')

    else:
        if (not ((snake[0][0] - snake[-2][0] == 1) and (snake[0][1] == snake[-2][1]))) and (not((snake[0][0] - snake[1][0] == 1) and (snake[0][1] == snake[1][1]))):
            move.add('u')

    #down
    if snake[0][0] == y:
        if not ((snake[1][0] == 0) and (snake[0][1] == snake[1][1])):
            move.add('d')

    else:
        if (not ((snake[0][0] - snake[-2][0] == -1) and (snake[0][1] == snake[-2][1]))) and (not((snake[0][0] - snake[1][0] == -1) and (snake[0][1] == snake[1][1]))):
            move.add('d')

    #right
    if snake[0][1] == x:
        if not ((snake[1][1] == 0) and (snake[0][0] == snake[1][0])):
            move.add('r')

    else:
        if (not ((snake[0][1] - snake[-2][1] == -1) and (snake[0][0] == snake[-2][0]))) and (not((snake[0][1] - snake[1][1] == -1) and (snake[0][0] == snake[1][0]))):
            move.add('r')
    #left
    if snake[0][1] == 0:
        if not ((snake[1][1] == x) and (snake[0][0] == snake[1][0])):
            move.add('l')

    else:
        if (not ((snake[0][1] - snake[-2][1] == 1) and (snake[0][0] == snake[-2][0]))) and (not((snake[0][1] - snake[1][1] == 1) and (snake[0][0] == snake[1][0]))):
            move.add('l')

After we detect in which directions the snake can move, we have to create the new snake. If it ate an apple we just need to add a new part to its body which is its new head. Its coordinates are calculated based on the direction. If the snake did not eat an apple, we have to add a new head, shift all parts of its body(for example, the old head now becomes second part of its body) and delete its tail. 

In [604]:
def make_new_snake(move, node, eat, y, x):
    if move == 'u':
        return up_move(node, eat, y)
    if move == 'd':
        return down_move(node, eat, y)
    if move == 'l':
        return left_move(node, eat, x)
    if move == 'r':
        return right_move(node, eat, x)

In [605]:
def up_move(node, eat, y):
    snake = []
    head = node.snake[0]
    if head[0] == 0:
        snake.append([y, head[1]])
    else:
        snake.append([head[0] - 1, head[1]])
    for i in range(len(node.snake) - 1):
        snake.append(node.snake[i])
    if eat:
        snake.append(node.snake[-1])
    return snake
def down_move(node, eat, y):
    snake = []
    head = node.snake[0]
    if head[0] == y:
        snake.append([0, head[1]])
    else:
        snake.append([head[0] + 1, head[1]])
    for i in range(len(node.snake) - 1):
        snake.append(node.snake[i])
    if eat:
        snake.append(node.snake[-1])
    return snake

def left_move(node, eat, x):
    snake = []
    head = node.snake[0]
    if head[1] == 0:
        snake.append([head[0], x])
    else:
        snake.append([head[0], head[1] - 1])
    for i in range(len(node.snake) - 1):
        snake.append(node.snake[i])
    if eat:
        snake.append(node.snake[-1])
    return snake

def right_move(node, eat, x):
    snake = []
    head = node.snake[0]
    if head[1] == x:
        snake.append([head[0], 0])
    else:
        snake.append([head[0], head[1] + 1])
    for i in range(len(node.snake) - 1):
        snake.append(node.snake[i])
    if eat:
        snake.append(node.snake[-1])
    return snake

When goal state is reached, the search will stop and we print the solution. The path we need to travel, length of it, total number of states and number of unique states are shown as result. 

In [606]:
def print_result(node):
    path = ''
    count = 0
    while node.first != 'first':
        path = node.move + ',' + path
        node = node.par
        count += 1
    print(path[:-1])
    print(count)

**Now we explain each Search strategy**

**BFS**

Starting with the initial node, we add it to queue(which is our frontier). In each step, we pop the first element of queue. First we check if we can eat an apple. Then we check if we are in the goal state. For each move that this node(parent node) can do, we define a child node with new state different from parent and other children(because the direction of action is different, so the snake will be different). We have an explored set named *exp* which includes all the states we have visited or are in the queue. If the state of child node is not visited, we add it to the queue and add its state to exp. Below is the pseudo code of algorithm:


function BREADTH-FIRST-SEARCH(problem) returns a solution, or failure
node ←a node with STATE = problem.INITIAL-STATE, PATH-COST = 0
if problem.GOAL-TEST(node.STATE) then return SOLUTION(node)
frontier ←a FIFO queue with node as the only element
explored ←an empty set
loop do
if EMPTY?( frontier) then return failure
node←POP( frontier ) /* chooses the shallowest node in frontier */
add node.STATE to explored
for each action in problem.ACTIONS(node.STATE) do
child ←CHILD-NODE(problem, node, action)
if child .STATE is not in explored or frontier then
if problem.GOAL-TEST(child .STATE) then return SOLUTION(child )
frontier ←INSERT(child , frontier )

In [607]:
def BFS(node, size, y, x):
    exp = set()
    q = queue.Queue()
    q.put(node)
    head = node.snake[0]
    exp.add(node.state)
    all_states = 1
    while True:
        if q.empty():
            print('There is no way to eat all!')
            return
        node = q.get()
        num = node.num
        food1 = node.food1.copy()
        food1 = list(food1)
        food2 = node.food2.copy()
        food2 = list(food2)
        food1_t = tuple(food1)
        food2_t = tuple(food2)
        head = node.snake[0]
        [eat, num] = eat_apple(head, food1, food2, num)
        if test_goal(num):
            print_result(node)
            print('All of visited states: ', all_states)
            print('Unique states: ', len(exp))
            break
        move = set()
        find_move(node.snake, move, y, x)
        for i in move:
            all_states += 1
            snake = make_new_snake(i, node, eat, y, x)
            state = (tuple(map(tuple, snake)), food1_t, food2_t)
            if state not in exp:
                new_node = Node(snake, i, node, num, food1, food2)
                new_node.state = state
                q.put(new_node)
                exp.add(state)

**IDS**

IDS algorithm is combination of BFS and DFS algorithm. It is optimal like BFS and has DFS memory consumption which is less than BFS.
It first call dfs search of depth 1. Then at each step it increases the depth until it finds goal state. 
In *dfs* we define a dictionary named opt for two purposes. First, we can save visited states. Second, if we have a state that we have already visited but now we reached to it with a lower path cost, we should replace it with the new one to get the optimal answer. The pseudo code is shown below:

function RECURSIVE-DLS(node, problem) returns a solution, or failure/cutoff
if problem.GOAL-TEST(node.STATE) then return SOLUTION(node)
else if limit = 0 then return False
else
for each action in problem.ACTIONS(node.STATE) do
child ←CHILD-NODE(problem, node, action)
result ←RECURSIVE-DLS(child , problem, limit − 1)


function ITERATIVE-DEEPENING-SEARCH(problem) returns a solution, or failure
for depth = 0 to∞ do
result ←RECURSIVE-DLS(problem, depth)
if result != cutoff then return result

In [608]:
def IDS(node, y, x):
    ans = False
    depth = 0
    count = 0
    length = 0
    while not ans:
        opt = dict()
        opt[node.state] = 0
        count += 1
        [ans, new_count] = dfs(node, depth, 0, y, x, opt, count)
        count += new_count
        depth += 1
        length = len(opt)
    print('All of visited state: ', count)
    print('Unique states: ', length)

In [609]:
def dfs(node, max_depth, depth, y, x, opt, count):
    if max_depth <= 0:
        return False, count
    head = node.snake[0]
    num = node.num
    food1 = node.food1.copy()
    food1 = list(food1)
    food2 = node.food2.copy()
    food2 = list(food2)
    food1_t = tuple(food1)
    food2_t = tuple(food2)
    [eat, num] = eat_apple(head, food1, food2, num)
    if test_goal(num):
        print_result(node)
        return [True, count]
    move = set()
    find_move(node.snake, move, y, x)
    for i in move:
        snake = make_new_snake(i, node, eat, y, x)
        state = (tuple(map(tuple, snake)), food1_t, food2_t)
        if (state not in opt) or (state in opt and opt[state] > depth):
            opt[state] = depth
            new_node = Node(snake, i, node, num, food1, food2)
            new_node.state = state
            [ans, new_count] = dfs(new_node, max_depth - 1, depth + 1, y, x, opt, count + 1)
            if ans:
                return True, count
   
    
    return False, count

**A***

In this algorithm, we need a heuristic fuction to estimate cost from this state to goal for each node. We implement this algorithm with two differnet heuristics. The first one finds the nearest apple to head of the snake and calculates their *Manhatan* distance. This heuristic is consistent because the actual path that the snake has to traverse is equal or more than the distance we estimate. so for each node h(node2) + path_cost(node1 to node2) >= h(node1) is true. As this heuristic is consistent, we can find the optimal answer with graph search.
The other heuristic could be similar to the first one. Instead of calculating distance to closest apple, we calculate the distance to the furthest apple. This heuristic is admissible because for each node the minimal cost to get to the goal state is to eat the last apple(assume we have eaten all other apples). So this heuristic is always less than or equal to actual cost. BEcause this heuristic is only admissible and not consistent and we use graph search, we cannot find an optimal path.
For implementing *A star* algorithm, we define a heap called *frontier* to store nodes that can be expanded. We define an explored set to avoid visiting duplicate states. We have a dictionary called *opt*. its keys are states which are states of nodes in *frontier* and its value is the f(node) = h(node) + g(node).
Each step we pop from the heap(which gives us the node with minimum f(node)) and we check if the snake can eat apple. Then we determine new states based on its move. For the new state that we get, we check if it is in explored set. If not, we check if this state is a key in our dictionary. If it is, it means it already exists in the *frontier*. But if the node in the *frontier* has the path cost more than the new state we found, it should be replace.
At the end if none of the conditions above is true, we add this node to *frontier*.
Below the pseudo code is shown.


make a frontier list containing only the starting node
   make an empty explored list
   while (the goal state has not been reached):
       consider the node with the lowest f score in the frontier list
       if (this node is our goal state) :
           we are finished 
       if not:
           put the current node in the explored list and look at all of its neighbors
           for (each neighbor of the current node):
               if (neighbor is in frontier list and has less f) :
                   replace the node with lower f           
               else if this neighbor is not in explored list:
                   add it to the frontier list and set its f

In [610]:
def heuristic(head, food1, food2, y, x, alpha):
    h = 0
    if len(food1) != 0 and len(food2) != 0:
        closest = min(min(food1), min(food2))
    else:
        if len(food1) != 0:
            closest = min(food1)
        else:
            closest = min(food2)
#heuristic 2- admissible
#     if len(food1) != 0 and len(food2) != 0:
#         closest = max(max(food1), max(food2))
#     else:
#         if len(food1) != 0:
#             closest = max(food1)
#         else:
#             closest = max(food2)

    dify = abs(head[0] - closest[0])
    if dify > (y // 2):
        dify = y - dify
    difx = abs(head[1] - closest[1])
    if difx > (x // 2):
        difx = x - difx
    h += dify + difx
    return h * alpha

In [611]:
def a_star(node, size, y, x, alpha):
    exp = set()
    opt = dict()
    frontier = []
    heapq.heappush(frontier, node)
    opt[node.state] = node.cost
    head = node.snake[0]
    count = 1
    while True:
        node = heapq.heappop(frontier)
        exp.add(node.state)
        del opt[node.state]
        num = node.num
        food1 = node.food1.copy()
        food1 = list(food1)
        food2 = node.food2.copy()
        food2 = list(food2)
        food1_t = tuple(food1)
        food2_t = tuple(food2)
        head = node.snake[0]
        [eat, num] = eat_apple(head, food1, food2, num)
        if test_goal(num):
            print_result(node)
            print('All of visited states: ', count)
            print('Unique states: ', len(exp) + len(opt))
            break
        move = set()
        find_move(node.snake, move, y, x)
        for i in move:
            count += 1
            snake = make_new_snake(i, node, eat, y, x)
            state = (tuple(map(tuple, snake)), food1_t, food2_t)
            if state not in exp:
                h = heuristic(snake[0], food1, food2, y, x, alpha)
                f = h + node.cost - node.h + 1
                if state in opt and opt[state] > f:
                    opt[state] = f
                    length = len(frontier)
                    for j in range(length):
                        if frontier[j].state == state:
                            frontier[j].cost = f
                            frontier[j].move = i
                            frontier[j].par = node
                            break
                if state not in opt:
                    new_node = Node(snake, i, node, num, food1, food2)
                    new_node.state = state
                    new_node.set_cost(f, h)
                    heapq.heappush(frontier, new_node)
                    opt[state] = f

In [612]:
filename = input()
snake = []
food1 = []
food2 = []
[size, y, x] = get_input(filename, snake, food1, food2)
food1.sort()
food2.sort()
node = initial_state(snake, size, food1, food2)

test3.txt


In [613]:
tic = time.time()
BFS(node, size, y, x)
toc = time.time()
print('time %f s' % ((toc - tic)))

u,r,d,d,d,r,d,r,r,d,d,r,r,r,u,r,r,d,l,l,u,u,l,l,l
25
All of visited states:  652320
Unique states:  287438
time 14.580391 s


In [614]:
tic = time.time()
IDS(node, y, x)
toc = time.time()
print('time %f s' % ((toc - tic)))

u,r,d,d,d,r,d,r,r,d,d,r,r,r,u,r,r,d,l,l,u,u,l,l,l
25
All of visited state:  268435454
Unique states:  277461
time 63.684227 s


In [615]:
tic = time.time()
h = heuristic(snake[0], food1, food2, y, x, 1)
node.set_cost(0, h)
a_star(node, size, y, x, 1)
toc = time.time()
print('time %f s' % ((toc - tic)))

r,u,d,d,d,r,r,d,r,r,u,l,d,d,r,d,r,r,u,l,d,r,u,r,r
25
All of visited states:  172977
Unique states:  84548
time 4.357950 s


In [616]:
tic = time.time()
h = heuristic(snake[0], food1, food2, y, x, 2)
node.set_cost(0, h)
a_star(node, size, y, x, 2)
toc = time.time()
print('time %f s' % ((toc - tic)))

r,u,r,d,d,d,r,r,d,r,d,l,u,r,r,r,r,r,d,l,l,d,l,d,r,u
26
All of visited states:  100357
Unique states:  49585
time 2.916221 s


In [617]:
tic = time.time()
h = heuristic(snake[0], food1, food2, y, x, 5)
node.set_cost(0, h)
a_star(node, size, y, x, 5)
toc = time.time()
print('time %f s' % ((toc - tic)))

r,u,r,d,d,d,d,r,r,u,l,d,r,d,r,r,r,d,r,u,r,d,l,d,l,u
26
All of visited states:  19173
Unique states:  10314
time 0.537904 s


**Results of Test 1**

|             |Distance  |Path                     |Number of total States           |Number of Unique States|Time|
|-------------|----------|-------------------------|-------------------------------- |-----------------------|----|
|BFS          |12        |d,l,d,d,r,r,r,d,r,d,d,r  |13504                            |6435                   |0.29|
|IDS          |12        |d,l,d,d,r,r,r,d,r,d,d,r  |262142                           |6140                   |0.86|
|A star(1)    |12        |d,l,r,d,l,u,u,u,l,l,u,l  |6396                             | 3676                  |0.14|
|A star(2)    |12        |l,d,u,l,u,u,l,l,l,u,u,l  |5070                             |2987                   |0.22    |
|W A star(2)  |13        |d,l,u,l,u,r,d,d,d,d,r,r,r|3308                             |1874                   |0.07|
|W A star(5)  |13        |d,l,u,r,d,l,d,d,r,r,r,d,r|545                              |296                    |0.01|

**Results of Test 2**

|             |Distance  |Path                                         |
|-------------|----------|---------------------------------------------|
|BFS          |15        |u,r,d,l,l,u,u,u,u,l,u,l,l,l,l                |
|IDS          |15        |u,r,d,l,l,u,u,u,u,l,u,l,l,l,l                |
|A star(1)    |15        |r,l,l,u,r,u,l,u,u,l,l,l,l,u,l                |
|A star(2)    |15        |r,l,l,u,r,u,l,l,u,u,l,l,u,l,l                |                                 
|W A star(2)  |17        |r,u,l,l,d,l,l,u,u,u,u,r,u,l,l,l,l            |
|W A star(5)  |23        |l,r,r,r,r,r,r,r,u,u,u,u,u,r,r,r,d,r,d,d,d,r,r|



|             |Number of total States           |Number of Unique States|Time |
|-------------|-------------------------------- |-----------------------|-----|
|BFS          |151802                           |66057                  |3.14 |
|IDS          |262142                           |57938                  |11.49|
|A star(1)    |34404                            |18427                  |0.76 |
|A star(2)    |39764                            |19651                  |1.07 |
|W A star(2)  |80806                            |36901                  |2.37 |
|W A star(5)  |111327                           |43167                  |2.92 |

**Results of Test 3**

|             |Distance  |Path                                                             |
|-------------|----------|-----------------------------------------------------------------|
|BFS          |25        |u,r,d,d,d,r,d,r,r,d,d,r,r,r,u,r,r,d,l,l,u,u,l,l,l                |
|IDS          |25        |u,r,d,d,d,r,d,r,r,d,d,r,r,r,u,r,r,d,l,l,u,u,l,l,l                |
|A star(1)    |25        |r,u,d,d,d,r,r,d,r,r,u,l,d,d,r,d,r,r,u,l,d,r,u,r,r                |
|A star(2)    |25        |r,u,d,d,d,r,r,r,d,l,u,r,d,r,d,r,r,d,l,d,r,u,u,r,r                   |                          |W A star(2)  |26        |r,u,r,d,d,d,r,r,d,r,d,l,u,r,r,r,r,r,d,l,l,d,l,d,r,u              |
|W A star(5)  |26        |r,u,r,d,d,d,d,r,r,u,l,d,r,d,r,r,r,d,r,u,r,d,l,d,l,u              |


|             |Number of total States           |Number of Unique States|Time |
|-------------|-------------------------------- |-----------------------|-----|
|BFS          |652320                           |287438                  |14.53 |
|IDS          |268435454                           |277461                  |67.76|
|A star(1)    |172977                            |84548                  |4.57 |
|A star(2)    |182790                            |87353                       | 5.23    |
|W A star(2)  |100357                            |49585                  |3.12|
|W A star(5)  |19173                           |10314                  |0.52 |

Informed search strategies have better time complexities in comparison with uniformed search strategies. They visit fewer nodes but sometimes it is difficult to find a heuristic and it has more cost for us. BFS finds solution faster but it consumes more memory while IDS needs more time to find the solutin and has a better memory consumption.
In A* if our heuristic is consistent, we can find the optimal solution with graph search. If the heuristic is admissible, the optimal solution cannot be found with graph search and we need to implement tree search which does not care about visited nodes. So, it is slower. 
Weighted A* is used to find a solution faster at the expense of optimality. It reduces the impact of g-cost and it begins with a much flatter search. if we choose α = 2 the result is shorter than α = 5 but it is slower. Our heuristic for each step is to calculate distance between head of the snake and the closest apple. When we use α = 5 this value may get closer to the actual cost(because in fact we have to eat all apples not one of them) and the closer our assumption to the actual cost, the faster our algorithm. According to this, α = 5 is faster than α = 2. α = 2 is closer to the optimal solution because our heuristic is changed less and it can be admissible in many cases. 

## Conclusion
### Different search strategies can be used based on our resources. If we have limit in memory, IDS is a better option than BFS. When we want to find a solution and it is not so important to find the optimal one but we want to find it very fast, weighted A* can be used. The better our heuristic, the faster our A* algorithms will work. We have to find heuristics which are close to actual cost and are less than it. Sometimes finding these heurisristics have more cost for us.

## References
1. *Russell, Stuart J. (2018). Artificial intelligence a modern approach. Norvig, Peter (4th ed.)*
2. *https://brilliant.org/wiki/a-star-search/*
